# Best available pixel composite

In this notebook a monthly composite image is created based on the Best Available Pixel (BAP) method in OpenEO. This is a scientifically sound method of generating composites, which minimizes the mixing of pixels from different observations. Also for each pixel, all bands will be taken from a single observation. As a result, the spectral signature in the composite is observed rather than being a combination of multiple observations.

We refer to the scientific references for a full discussion of this method, but can generally recommend it both for its correctness and performance. We do still observe some residual cloud (shadow) in certain cases, which can be attributed to the quality of the SCL band. 

The BAP score is a weighted average of three (four) scores:

- **Distance-to-Cloud Score**: Pixels that are clouds are given score 0. Pixels that are more than 150 pixels - calculated with the Manhattan distance measure, assuming a resolution of 20m - away from a cloud pixel are given score 1. The pixels in between are given a score versus distance-to-cloud that follows a Gaussian shape.
- **Coverage Score**: Per date, the percentage of all pixels that are classified as a cloud over the entire spatial extent is calculated. The Coverage Score is then equal to 1 - the cloud percentage.
- **Date Score**: In order to favor pixels that are observed in the middle of a month, a date score is calculated, which follows a Gaussian shape. I.e. the largest scores are given for days in the middle of the month, the lowest scores are given for days at the beginning and end of the month. 

The final BAP score is a weighted average of the three aforementioned scores. The weights are 1, 0.5 and 0.8 for the Distance-to-Cloud, Coverage and Date Score respectively. 

The current implementation and the various parameters are still subject to change, so please do evaluate the correctness when using this method in your project. 

### References 

The approach described here was originally applied to Landsat, and adapted here for Sentinel-2. 

#### BAP composites assessment

Francini, S., Hermosilla, T., Coops, N. C., Wulder, M. A., White, J. C., & Chirici, G. (2023). An assessment approach for pixel-based image composites. ISPRS Journal of Photogrammetry and Remote Sensing, 202, 1–12, [doi:10.1016/j.isprsjprs.2023.06.002](https://doi.org/10.1016/j.isprsjprs.2023.06.002)

#### BAP process background:

White, J.C.; Wulder, M.A.; Hobart, G.W.; Luther, J.E.; Hermosilla, T.; Griffiths, P.; Coops, N.C.; Hall, R.J.; Hostert, P.; Dyk, A.; et al. Pixel-based image compositing for large-area dense time series applications and science. Can. J. Remote Sens. 2014, 40, 192–212, [doi:10.1080/07038992.2014.945827](https://doi.org/10.1080/07038992.2014.945827).


## Loading Sentinel-2 scene classification

The score is calculated based on the 'SCL' band only. This is very efficient because it's a lower resolution (20m) band, which is relatively quick to read.
All other bands are read based on the score, which allows openEO to avoid loading data that is not used in the composite.

First a sample period and region are defined. It is necessary to specify your region of interest as a Polygon.

In [2]:
import geopandas as gpd

gdf = gpd.read_file('test_area.geojson')
gdf = gdf.to_crs(epsg=4326)
area = eval(gdf.to_json())

In [3]:
temporal_extent = ["2022-07-01", "2022-07-31"]
max_cloud_cover = 70
spatial_resolution = 20

Note that the Python code behind the calculation of the BAP scores can be found in utils_BAP.py. All functions are defined there, and are loaded in here.

In [4]:
import openeo
from openeo.processes import if_, is_nan
from utils_BAP import (calculate_cloud_mask, calculate_cloud_coverage_score,
                           calculate_date_score, calculate_distance_to_cloud_score,
                           calculate_distance_to_cloud_score, aggregate_BAP_scores,
                           create_rank_mask)

In [5]:
c=openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


Load in the SCL band from Sentinel-2 which will be used to calculate all cloud-related scores.

In [6]:
from openeo.api.process import Parameter

temporal_parameter = Parameter.temporal_interval("temporal_extent",description="Temporal extent specified as two-element array with start and end date/date-time.")
geojson_parameter = Parameter.geojson("geometry",description="Geometries specified as GeoJSON object.")
#bands_parameter = Parameter.array("bands",description="Sentinel-2 L2A bands to include in the composite.",item_schema=[],default=["B04","B03","B02"],optional=True)
bands_parameter = Parameter.array(
    "bands",
    description="Sentinel-2 L2A bands to include in the composite.",
    item_schema={"type": "string", "enum": [
        "B02",
        "B03",
        "B04",
        "B05",
        "B06",
        "B07",
        "B08",
        "B8A",
        "B11",
        "B12"
    ]},
    default=["B04", "B03", "B02"],
    optional=True
)
bands_parameter.schema["minItems"] = 1

# Max cloud cover parameter
max_cloud_cover_parameter = Parameter.integer(
    "max_cloud_cover",
    description="Maximum cloud cover percentage allowed.",
    optional=True,
    default=70
)
max_cloud_cover_parameter.schema["minimum"] = 0
max_cloud_cover_parameter.schema["maximum"] = 100

scl = c.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=temporal_parameter,
    bands=["SCL"],
    max_cloud_cover=max_cloud_cover_parameter
).resample_spatial(spatial_resolution).filter_spatial(geojson_parameter)

scl = scl.apply(lambda x: if_(is_nan(x), 0, x))

Create a binary cloud mask, which gives 1 if a pixel is classified as cloud by SCL, an 0 otherwise.

In [7]:
cloud_mask =  calculate_cloud_mask(scl)

## Coverage score

First of all, the coverage score is calculated, which is equal to 1 - the percentage of pixels in the spatial_extent that are classified as cloud in SCL.

- A polygon is created that is slighty larger than the original spatial_extent
- The cloud coverage percentage is calculated by taking the average of the cloud binary (defined above), by using the `aggregate_spatial` process
- Since the `aggregate_spatial` process results in vectordata, the data is rasterized again to the original dimensions, using the (experimental) `vector_to_raster` process
- The coverage score is then equal to 1 - the cloud coverage percentage. I.e. a smaller cloud coverage percentage indicates a larger score

In [8]:
coverage_score = calculate_cloud_coverage_score(cloud_mask, area, scl)

## Date Score

Both the date and distance-to-cloud score are calculated with a function in Bap_utils.py

In [9]:
date_score = calculate_date_score(scl)

c:\Users\holemant\OneDrive - VITO\Documents\Engineering\OpenEO\.venv\Lib\site-packages\openeo\api\process.py:48: UserWarning: Parameter without description: using name as description.
  warnings.warn("Parameter without description: using name as description.")


## Distance to Cloud Score

Finally, the Distance to Cloud Score is calculated by applying a Gaussian kernel to the binary.
The kernel size, defined below, indicates the maximum distance from a cloud, above which a pixel is automatically assigned score 1. In case of a kernel size of 151, the maximum distance is 150 pixels. The kernel size is 151 for a resolution of 20m and will be rescaled accordingly.

In [10]:
dtc_score = calculate_distance_to_cloud_score(cloud_mask, spatial_resolution)

## Aggregating Scores

Then, the scores are aggregated together, by using a weighted average. As for now the coverage score is excluded. To make sure that pixels which contain no data (SCL score 0) are not selected, they are explicitly masked out of the final score. B01 score is excluded.

In [11]:
weights = [1, 0.1, 0.5]
score = aggregate_BAP_scores(dtc_score, date_score, coverage_score, weights)
score = score.mask(scl.band("SCL") == 0)

## Masking and Compositing

Next, a mask is created. This serves to mask every pixel, except the one with the highest score, for each month.

In [12]:
rank_mask = create_rank_mask(score)

Firstly, some bands of interest from Sentinel-2 are loaded. They are then masked by the BAP mask constructed above. Then they are aggregated per month, to obtain a composite image per month. By using the "first" process as an aggregator, the situation where there are potentially more than one days in a month selected by the algorithm is immediately handled.

In [13]:
rgb_bands = c.load_collection(
    "SENTINEL2_L2A",
    temporal_extent = temporal_parameter,
    bands = bands_parameter,
    max_cloud_cover=max_cloud_cover_parameter
).filter_spatial(geojson_parameter)

compositeRGB = rgb_bands.mask(rank_mask).mask(cloud_mask).aggregate_temporal_period("month","first")

In [14]:
compositeRGB.flat_graph()

{'loadcollection1': {'process_id': 'load_collection',
  'arguments': {'bands': {'from_parameter': 'bands'},
   'id': 'SENTINEL2_L2A',
   'properties': {'eo:cloud_cover': {'process_graph': {'lte1': {'process_id': 'lte',
       'arguments': {'x': {'from_parameter': 'value'},
        'y': {'from_parameter': 'max_cloud_cover'}},
       'result': True}}}},
   'spatial_extent': None,
   'temporal_extent': {'from_parameter': 'temporal_extent'}}},
 'filterspatial1': {'process_id': 'filter_spatial',
  'arguments': {'data': {'from_node': 'loadcollection1'},
   'geometries': {'from_parameter': 'geometry'}}},
 'loadcollection2': {'process_id': 'load_collection',
  'arguments': {'bands': ['SCL'],
   'id': 'SENTINEL2_L2A',
   'properties': {'eo:cloud_cover': {'process_graph': {'lte2': {'process_id': 'lte',
       'arguments': {'x': {'from_parameter': 'value'},
        'y': {'from_parameter': 'max_cloud_cover'}},
       'result': True}}}},
   'spatial_extent': None,
   'temporal_extent': {'from_param

In [15]:
from openeo.rest.udp import build_process_dict
import json 

spec = build_process_dict(
    process_id="bap_composite",
    summary="Best Available Pixel (BAP) composite from Sentinel-2 L2A data at 20m resolution.",
    description="# Description\n\nThis algorithm generates a Sentinel-2 based composite for a selected area and temporal extent. By default, the resolution of the output is 20 meters.\n\nThe used compositing method is the \"Best-Available-Pixel (BAP)\" method, which selects the pixel with the highest BAP Score for each pixel location and within the time window. The method falls under the 'rank composite' category, and ensures that selected spectral band values for any individual pixel all come from the same observation.\n\nThe nature of the pixel selection - i.e. only choose the best pixel within each time window - guarantees that each pixel in the final composite is an actual observed value, rather than a statistical aggregation of multiple observations within the time window.\n\nThe BAP score is a weighted average of three scores:\n\n- **Distance-to-Cloud Score**: Pixels that are clouds are given score 0. Pixels that are more than 150 pixels - calculated with the Manhattan distance measure, assuming a resolution of 20m - away from a cloud pixel are given score 1. The pixels in between are given a score versus distance-to-cloud that follows a Gaussian shape.\n- **Coverage Score**: Per date, the percentage of all pixels that are classified as a cloud over the entire spatial extent is calculated. The Coverage Score is then equal to 1 - the cloud percentage.\n- **Date Score**: In order to favor pixels that are observed in the middle of a month, a date score is calculated, which follows a Gaussian shape. I.e. the largest scores are given for days in the middle of the month, the lowest scores are given for days at the beginning and end of the month.\n\nThe final BAP score is a weighted average of the three aforementioned scores. The weights are 1, 0.5 and 0.8 for the Distance-to-Cloud, Coverage and Date Score respectively.\n\n\n\n# Performance characteristics\n\nThe method is computationally efficient, as it only requires the SCL band to determine the rank score. Loading of other bands can be minimized to read only selected observations.\n\n\n# Examples\n\n## 1-month composite over Denmark\n\nA complete example of creating a monthly BAP composite over a time window of one month, including STAC metadata is shown here:\n\nhttps://s3.waw3-1.cloudferro.com/swift/v1/APEx-examples/bap_composite_denmark/collection.json\n\nThree bands were extracted over an area of ca. 60 km².\n\nThe processing platform reported these usage statistics for the example:\n\n```\nCredits: 14 \nCPU usage: 7,214.715 cpu-seconds\nWall time: 577 seconds\nInput Pixel 641.575 mega-pixel\nMax Executor Memory: 3.262 gb\nMemory usage: 25,839,702.134 mb-seconds\nNetwork Received: 71,733,702,645 b\n```\n\nThe relative cost is 0.2 CDSE platform credits per km² for a 1 month input window for three bands.\nThe cost per input pixel is 0.022 credits per megapixel.\n\n## 12-month composite over Denmark\n\nIn a second example, a longer time window was tested, generating 12 3-band results. Here we see a lower cost per km², but a similar cost per input pixel.\n\n```\nCredits: 88\nCPU usage: 54,250.901 cpu-seconds\nWall time: 3,321 seconds\nInput Pixel: 5,784.704 mega-pixel\nMax Executor Memory: 4.401 gb\nMemory usage: 231,391,081.839 mb-seconds\nNetwork Received: 761,633,043,483 b\n```\n\nThe relative cost is 0.12 CDSE platform credits per km² for a 12 month input window.\nThe cost per input pixel is 0.015 credits per megapixel.\n\n# Literature references\n\nThe approach described here was originally applied to Landsat, and adapted here for Sentinel-2.\n\n#### BAP composites assessment\n\nFrancini, S., Hermosilla, T., Coops, N. C., Wulder, M. A., White, J. C., & Chirici, G. (2023). An assessment approach for pixel-based image composites. ISPRS Journal of Photogrammetry and Remote Sensing, 202, 1–12, [doi:10.1016/j.isprsjprs.2023.06.002](https://doi.org/10.1016/j.isprsjprs.2023.06.002)\n\n#### BAP process background:\n\nWhite, J.C.; Wulder, M.A.; Hobart, G.W.; Luther, J.E.; Hermosilla, T.; Griffiths, P.; Coops, N.C.; Hall, R.J.; Hostert, P.; Dyk, A.; et al. Pixel-based image compositing for large-area dense time series applications and science. Can. J. Remote Sens. 2014, 40, 192–212, [doi:10.1080/07038992.2014.945827](https://doi.org/10.1080/07038992.2014.945827).\n\n# Known limitations\n\n- Due to the nature of the OpenEO implementation of this algorithm, the spatial extent has to be in inline GeoJSON format.\n- The BAP score is fully dependent on the quality of the SCL band. Wrongly classified pixels may therefore receive a higher score than expected.\n- The method is efficient up to 50x50km areas. For larger areas of interest, we recommend splitting the area into smaller tiles.\n<!-- For different areas of interest, the user may need to tweak the relative weights of the three scores in BAP. This tweaking can be a lengthy procedure that is largely based on trial-and-error. Currently not applicable as the weights are not parametrized in the first version. -->\n\n# Known artifacts\n\nResidual cloud artifacts may be present in the composite, especially for smaller time windows or during cloudy seasons. The cloud artifacts are caused by the limited capabilities of the default Sentinel-2 cloud detection mechanism to correctly identify all clouds.\nThe picture below shows a composite image with some cloud shadow pixels remaining. Because these pixels were not classified as cloud shadow in the SCL band, they are not masked out in the final composite.\n\n![cloud-shadow-artifacts-bap.png](./cloud-shadow-artifacts-bap.png)",
    process_graph=compositeRGB,
    parameters=[geojson_parameter,temporal_parameter,bands_parameter,max_cloud_cover_parameter],
)

with open("example.json", "w") as f:
    json.dump(spec, f, indent=2)